# Consultas e Seleções

Vamos explorar o uso de SQL dentro do Apache Spark

In [60]:
from pyspark.sql import SparkSession

In [62]:
spark = SparkSession.builder \
      .master("local[*]") \
      .appName("postech") \
      .getOrCreate()

## Spark SQL

In [63]:
df = spark.sql('SELECT "ok" as Status')
df.show()

+------+
|Status|
+------+
|    ok|
+------+



Importando os dados para realizarmos consultas e seleções usando SQL

In [7]:
df = spark.read.csv('data/cereal.csv', header=True, inferSchema=True)
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



Precisamos antes de tudo, criar uma View temporária em nossa sessão para realizar as operações SQL

In [8]:
df.createOrReplaceTempView('cereal')

In [9]:
cereal = spark.sql('SELECT * FROM cereal')
cereal.show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [11]:
spark.sql("SELECT * FROM cereal WHERE type = 'C'").show()
# Equivalente a df.where(df.type == 'C').show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [15]:
spark.sql("SELECT COUNT(1) AS Total FROM cereal WHERE type = 'C'").show()
# Equivalente a df.filter(df.type == 'C').count() ou spark.sql("SELECT * FROM cereal WHERE type = 'C'").count()

+-----+
|Total|
+-----+
|   74|
+-----+



É possível usarmos cosultas SQL normalmente (mesmo com clausulas mais avançadas)

In [49]:
spark.sql(''' 
SELECT
    mfr,
    AVG(calories) AS avg_calories,
    CAST(AVG(protein) AS DECIMAL(10,3)) AS avg_protein,
    ROUND(SUM(weight), 1) AS total_weight,
    COUNT(1) AS total,
    CASE WHEN COUNT(1) > 10 THEN 'Mais que 10' ELSE 'Menos que 10' END AS contagem
FROM 
    cereal
WHERE
    shelf > 2
GROUP BY 
    mfr
HAVING 
    avg_calories > 100
ORDER BY
    total DESC
''').show()

+---+------------------+-----------+------------+-----+------------+
|mfr|      avg_calories|avg_protein|total_weight|total|    contagem|
+---+------------------+-----------+------------+-----+------------+
|  K|             107.5|      2.917|        13.5|   12| Mais que 10|
|  G|114.44444444444444|      2.667|        10.1|    9|Menos que 10|
|  P|             110.0|      3.000|         6.6|    6|Menos que 10|
|  R|             127.5|      3.000|         4.0|    4|Menos que 10|
+---+------------------+-----------+------------+-----+------------+



Podemos trabalhar com JOINs também

In [58]:
df_agg = spark.sql(''' 
SELECT
    mfr,
    AVG(calories) AS avg_calories,
    CAST(AVG(protein) AS DECIMAL(10,3)) AS avg_protein,
    ROUND(SUM(weight), 1) AS total_weight,
    COUNT(1) AS total,
    CASE WHEN COUNT(1) > 10 THEN 'Mais que 10' ELSE 'Menos que 10' END AS contagem
FROM 
    cereal
WHERE
    shelf > 2
GROUP BY 
    mfr
HAVING 
    avg_calories > 100
ORDER BY
    total DESC
''')

df_agg.createOrReplaceTempView('cereal_agg')

In [59]:
spark.sql(''' 
SELECT
    cereal.*,
    agg.contagem
FROM cereal
LEFT JOIN cereal_agg AS agg
    ON cereal.mfr = agg.mfr
''').show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+------------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|    contagem|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+------------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|        NULL|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|        NULL|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505| Mais que 10|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912| Mais que 10|
|      Almond Delight|  R|   C|     110|      2|